In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import timedelta

from google.colab import files
#src = list(files.upload().values())[0]    # add a new lib from computer
#open('mylib.py','wb').write(src)
import mylib

import cifar10

cifar10.download()

print(cifar10.load_class_names())

# get images, classes and labels of "train & test"
train_img, train_cls, train_labels = cifar10.load_training_data()
test_img, test_cls, test_labels = cifar10.load_test_data()

# How many images are that we got
print(len(train_img), len(test_img))    #50k 10k

x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y_true = tf.placeholder(tf.float32, [None, 10])
pkeep = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool)  #eğitim aşamasında mı? test aşamasında mı? tutucak

###DATA AUGMENTATION###
#Data Augmentation
def pre_process_image(image):
  image = tf.image.random_flip_left_right(image) #resmi ters çevir
  image = tf.image.random_hue(image, max_delta=0.05) #resmin renkleriyle oyna
  image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
  image = tf.image.random_brightness(image, max_delta=0.2)
  image = tf.image.random_saturation(image, lower=0.0, upper=2.0) #doygunlukla oyna
  
  image = tf.minimum(image, 1.0)  # güvenlik önlemi
  image = tf.maximum(image, 0.0)
  return image
 
#pre_process_image'ı tüm batch'e uygula 
def pre_process(images):
  images = tf.map_fn(lambda image: pre_process_image(image), images)
  return images
  
with tf.device('/cpu:0'):      # bu kısmı, cpu'da çalıştır(performans amaçlı)
  distorted_images = pre_process(images=x)
### end of DATA AUGMENTATION###
  
  
###NORMALIZATION###
                                    #scope isimle alakalı
def batch_normalization(input, phase, scope):
  return tf.cond(phase,    #if phase=true(eğitim)
                 lambda: tf.contrib.layers.batch_norm(input, decay=0.99, is_training=True,
                                                     updates_collections=None, center=True, scope=scope),  #center=True ortalamasını sağlıyor
                 lambda: tf.contrib.layers.batch_norm(input, decay=0.99, is_training=False,
                                                     updates_collections=None, center=True, scope=scope, reuse=True))  #reuse: eğitilen w'leri tekrar kullanmak için
  
  
  
###end of NORMALIZATION###
  
  
#Convulasyon Layer 
def conv_layer(input, size_in, size_out, scope, use_pooling=True):
                                  # 3x3 'lük filtre
  w = tf.Variable(tf.truncated_normal([3, 3, size_in, size_out], stddev=0.1))
  b = tf.Variable(tf.constant(0.1, shape=[size_out]))    #shape is size of bias
                               #atılımlar
  conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding='SAME') + b
  conv_bn = batch_normalization(conv, phase, scope)   #Normalization
  y = tf.nn.relu(conv_bn)    #relu, aktivasyon fonksiyonumuz
  
  if use_pooling:         #boyut 2x2 olduğundan atılım da 2x2 ki çakışma olmasın
    y = tf.nn.max_pool(y, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
  return y


#fully connected layer
def fc_layer(input, size_in, size_out, scope, relu=True,dropout=True, batch_norm=False):
  w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1))
  b = tf.Variable(tf.constant(0.1, shape=[size_out]))
  logits = tf.matmul(input, w) + b
  
  if batch_norm:    #Normalization
    logits = batch_normalization(logits, phase, scope)
  
  if relu:
    y = tf.nn.relu(logits)
    if dropout:
      y = tf.nn.dropout(y, pkeep)
    return y
  else:
    return logits
  
# input, derinlik(veya kanal sayısı), kaç filtre olucak
conv1 = conv_layer(distorted_images, 3, 32, scope='conv1', use_pooling=True)   #işlem bittiğinde 16,16,32 olucak     scope='conv1', layer'ın ismini atadık
conv2 = conv_layer(conv1, 32, 64, scope='conv2', use_pooling=True)     #işlem bittiğinde 8,8,64 olucak
conv3 = conv_layer(conv2, 64, 64, scope='conv3', use_pooling=True)     #işlem bittiğinde 4,4,64 olucak

# conv layer'ı, fully connected layer'a bağlaman için düzleştirmen lazım
flattened = tf.reshape(conv3, [-1, 4*4*64])

fc1 = fc_layer(flattened, 4*4*64, 512, scope='fc1', relu=True, dropout=True, batch_norm=True)
fc2 = fc_layer(fc1, 512, 256, scope='fc2', relu=True, dropout=True, batch_norm=True)
#output layer
#fc3 = logits
logits = fc_layer(fc2, 256, 10, scope='fc_out', relu=False, dropout=False, batch_norm=False)
y = tf.nn.softmax(logits) # bu kez relu yerine softmax kullandık

y_pred_cls = tf.argmax(y, 1)   # tahmin edilen sınıfı tut (1=dimension)

xent = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_true)
loss = tf.reduce_mean(xent)

correct_prediction = tf.equal(y_pred_cls, tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.train.AdamOptimizer(5e-4).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# eğitim esnasında hafıza hatası alırsan burayı düşür
batch_size = 128

def random_batch():
  # rastgele bir index oluşturuyorum, bu index'teki verileri al
  index = np.random.choice(len(train_img),size=batch_size, replace=False)
  x_batch = train_img[index, :, :, :]
  y_batch = train_labels[index, :]
  
  return x_batch, y_batch
  
loss_graph = []

def training_step (iterations):
  start_time = time.time()   #fonk başladığı andaki zamanı al
  for i in range (iterations):
    x_batch, y_batch = random_batch()
    feed_dict_train = {x: x_batch, y_true: y_batch, pkeep: 0.5, phase: True}
    [_, train_loss] = sess.run([optimizer, loss], feed_dict=feed_dict_train)
    loss_graph.append(train_loss)
  
    if i % 100 == 0 :  
      acc = sess.run(accuracy, feed_dict = feed_dict_train)
      print('Iteration:', i, 'Training accuracy:', acc, 'Training loss:', train_loss)
  
  end_time = time.time()
  time_dif = end_time - start_time
  print("Time usage:", timedelta(seconds=int(round(time_dif))))
  
#test
batch_size_test = 256

def test_accuracy():
  num_images = len(test_img)
  cls_pred = np.zeros(shape=num_images, dtype = np.int)
  i=0
  
  while i < num_images:
    j = min(i+batch_size_test,num_images) #num_images'i geçmesin diye
    feed_dict = { x: test_img[i:j, :], y_true: test_labels[i:j, :], pkeep: 1, phase: False}
    cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)
    i = j #i'yi batch_size_test kadar arttırıp, b_s_t kadar veri alıyoruz
    
  correct = (test_cls == cls_pred)
  print('Test accuracy:',correct.mean())
  
  
  

training_step(10000)
test_accuracy()

'''
plt.plot(loss_graph,'k-')
plt.title('Loss Grafiği')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()
'''
  
  
  
  
  
  